In [ ]:
import pandas as pd
import os
from datetime import datetime
import numpy as np

In [ ]:
df = pd.read_csv('/Users/avacheevers/Desktop/clearcut_categories/SQL_input/MCT Oil.csv')
df['Month'] = pd.to_datetime(df['Month'])
category = 'MCT Oil'

In [ ]:
# finds year of most recent data
year = str(df.sort_values('Month', ascending = 0).Month[0])[:4]
last_year = str(int(year)-1)

In [ ]:
# finds month of most recent data
month = str(df.sort_values('Month', ascending = 0).Month[0])[5:7]

In [ ]:
if month in ['12', '01', '02']:
    to_calc = ['Q3Q4', 'Q2Q3', 'Q1Q2', 'Q4Q1']
elif month in ['03','04' ,'05']:
    to_calc = ['Q4Q1', 'Q3Q4', 'Q2Q3', 'Q1Q2']
elif month in ['06','07', '08']:
    to_calc = ['Q1Q2', 'Q4Q1', 'Q3Q4', 'Q2Q3']
elif month in ['09','10', '11']:
    to_calc = ['Q2Q3', 'Q1Q2', 'Q4Q1', 'Q3Q4']

In [ ]:
def find_mask(quarter):
    if quarter == 'Q1':
        mask = (df['Month'] >= year+'-01-01') & (df['Month'] < year+'-04-01')
    if quarter == 'Q2':
        mask = (df['Month'] >= year+'-04-01') & (df['Month'] < year+'-07-01')
    if quarter == 'Q3':
        mask = (df['Month'] >= last_year+'-07-01') & (df['Month'] < last_year+'-09-01')
    if quarter == 'Q4':
        mask = (df['Month'] >= last_year+'-09-01') & (df['Month'] < year+'-01-01')
    return mask

def rev_format(rev):
    if rev > 10000000:
        formatted = str(round(rev/1000000,1))+' MM'
    elif rev > 500000: 
        formatted = str(round(rev/1000000,2))+' MM'
    else:
        formatted = str(round(rev/1000,1))+' K'
        
    return formatted

In [ ]:
def calculate_fields(quarters):
    a_quarter_name = quarters[:2]
    b_quarter_name = quarters[2:]
    
    a_mask = find_mask(a_quarter_name)
    a = df.loc[a_mask]
    
    b_mask = find_mask(b_quarter_name)
    b = df.loc[b_mask]
    
    a_agg = a.groupby(['tagbrand']).agg({'Revenue': ['sum']})
    a_agg.columns = ["_".join(x) for x in a_agg.columns.ravel()]
    a_agg = a_agg.reset_index()
    
    b_agg = b.groupby(['tagbrand']).agg({'Revenue': ['sum']})
    b_agg.columns = ["_".join(x) for x in b_agg.columns.ravel()]
    b_agg = b_agg.reset_index()
    
    ba = b_agg.merge(a_agg, how = 'left', on = ['tagbrand'])
    ba.columns = ['tagbrand', 'rev_b', 'rev_a']
    
    ba.rev_b = round(ba.rev_b, 0)
    ba.rev_a = round(ba.rev_a, 0)
    
    ba['market_share_b_perc'] = round(100*(ba.rev_b / ba.rev_b.sum()),2)
    ba['market_share_a_perc'] = round(100*(ba.rev_a / ba.rev_a.sum()),2)
    ba['category_growth_perc'] = round(100*((ba.rev_b.sum() - ba.rev_a.sum()) /ba.rev_a.sum()),2)
    ba['brand_growth_perc'] = round(100*((ba.rev_b - ba.rev_a)/ba.rev_a),2)
    ba['growth_gap'] = ba.brand_growth_perc - ba.category_growth_perc
    ba['market_share_perc_change'] = round(100*((ba.market_share_b_perc-ba.market_share_a_perc) / ba.market_share_a_perc),2)

    ba = ba.sort_values('market_share_a_perc', ascending = 0)
    ba = ba.reset_index()
    ba['rank_a'] = (ba.index)+1
    
    ba = ba.sort_values('market_share_b_perc', ascending = 0)
    ba = ba.reset_index()
    ba['rank_b'] = (ba.index)+1
    
    ba['rev_b_formatted'] = ba.rev_b.apply(lambda x: rev_format(x))
    ba['rev_a_formatted'] = ba.rev_a.apply(lambda x: rev_format(x))
    
    ### hook 1 ### - last row is nan
    
    ba['market_share_below_tagbrand'] = ba.tagbrand.shift(-1)
    ba['brand_growth_below'] = ba.brand_growth_perc.shift(-1)
    ba['hook_1'] = 'Over the last quarter, your brand, '+ba.tagbrand+' generated $'+ba.rev_b_formatted+', representing '+ba.brand_growth_perc.apply(lambda x: str(x))+'% growth quarter-over-quarter in the '+category+' category. A close competitor of yours, '+ba.market_share_below_tagbrand+', grew by '+ba.brand_growth_below.apply(lambda x: str(round(x,2)))+'% in that same period. If the following information was available for all your competitors in the '+category+' category would that be helpful or did I miss the mark? '
    
    
    ### hook 2 ###
    
    ba['market_share_below_val'] = ba.market_share_b_perc.shift(-1)
    ba['market_share_below_val_a'] = ba.market_share_a_perc.shift(-1)
    
    ba.loc[((ba.market_share_b_perc - ba.market_share_a_perc) > 0) & ((ba.market_share_below_val - ba.market_share_below_val_a) > 0) & ((ba.market_share_b_perc - ba.market_share_a_perc)<(ba.market_share_below_val - ba.market_share_below_val_a)), 'hook_2'] = 'In '+b_quarter_name+ ' our analysis shows that '+ba.tagbrand+' market share increased by '+(ba.market_share_b_perc - ba.market_share_a_perc).apply(lambda x: str(round(x,2)))+ ' to ' +ba.market_share_b_perc.apply(lambda x: str(x))+'%, while '+ba.market_share_below_tagbrand+' grew '+(ba.market_share_below_val - ba.market_share_below_val_a).apply(lambda x: str(round(x,2)))+' to '+ba.market_share_below_val.apply(lambda x: str(x))+'%. What if we could help you gain back market share from a competitor who is encroaching on your brand and help make sure '+ba.market_share_below_tagbrand+' does not catch up?'
  
    
    ### hook 3 ###
    
    ba['market_share_above_val'] = ba.market_share_b_perc.shift(1).astype(str)
    ba['market_share_two_above_val'] = ba.market_share_b_perc.shift(2).astype(str)
    ba['market_share_three_above_val'] = ba.market_share_b_perc.shift(3).astype(str)
    
    ba.loc[ba.rank_b > 3, 'hook_3'] =  'In '+b_quarter_name+ ' our analysis indicates that your market share was '+ba.market_share_b_perc.apply(lambda x: str(x))+'% in the '+category+' category, while the 3 brands above you had shares of '+ba.market_share_above_val.apply(lambda x: str(x))+'%, '+ba.market_share_two_above_val.apply(lambda x: str(x))+'%, & '+ba.market_share_three_above_val.apply(lambda x: str(x))+'% respectively. Do you want us to help you catch up to them by growing your market share and by leveraging VMS data in the Amazon space?'
    
    
    ### hook 4 ###
    
    ba['rev_above_tagbrand'] = ba.tagbrand.shift(1)
    #ba['market_share_above_val'] = ba.market_share_b_perc.shift(1)
    
    ba.loc[((ba.market_share_above_val.astype(float) - ba.market_share_b_perc)>0) & (ba.rank_b>1),'hook_4'] = 'This quarter '+ba.tagbrand+' generated $'+ba.rev_b_formatted+ ' in the '+ category +' category. The brand ranked above you, '+ba.rev_above_tagbrand.apply(lambda x: str(x))+', has '+ (ba.market_share_above_val.astype(float) - ba.market_share_b_perc).apply(lambda x: str(round(x,2)))+'% more in market share in the '+category+' category. What if you could strategically outmaneuver your competition to gain market share using e-commerce business intelligence your competitors don’t have?'

    
    ### hook 5 ###

    ba['rev_above'] = ba.rev_b.shift(1)
    ba['rev_gap_from_brand_above'] = ba['rev_above'] - ba['rev_b']
    ba['rev_gap_from_brand_above'] = ba['rev_gap_from_brand_above'].apply(lambda x: rev_format(x))
    ba['rev_two_above'] = ba.rev_b.shift(2)
    ba['rev_gap_from_brand_two_above'] = ba['rev_two_above'] - ba['rev_b']
    ba['rev_gap_from_brand_two_above'] = ba['rev_gap_from_brand_two_above'].apply(lambda x: rev_format(x))
    ba['rev_three_above'] = ba.rev_b.shift(3)
    ba['rev_gap_from_brand_three_above'] = ba['rev_three_above'] - ba['rev_b']
    ba['rev_gap_from_brand_three_above'] = ba['rev_gap_from_brand_three_above'].apply(lambda x: rev_format(x))


    
    ba['rank_gap'] = ba['rank_b'] - ba['rank_a']
    ba['rev_two_above_tagbrand'] = ba.tagbrand.shift(2).astype(str)
    ba['rev_three_above_tagbrand'] = ba.tagbrand.shift(3).astype(str)
    
    
    ba.loc[(ba.rank_b > ba.rank_a) & (ba.rank_gap==1),'hook_5'] = 'Did you know that '+ba.rev_above_tagbrand+' took the #'+ (ba.rank_b-1).apply(lambda x: str(x)) + ' spot from you in '+ a_quarter_name + ' this year in the '+category+' category? Increasing quarterly sales by $'+ba.rev_gap_from_brand_above+' can help you retake your previous rank. What if we could help you gain back market share from a competitor who has recently surpassed your brand?'
    ba.loc[(ba.rank_b > ba.rank_a) & (ba.rank_gap==2),'hook_5'] = 'Did you know that '+ba.rev_two_above_tagbrand+' took the #'+ (ba.rank_b-2).apply(lambda x: str(x)) + ' spot from you in '+ a_quarter_name + ' this year in the '+category+' category? Increasing quarterly sales by $'+ba.rev_gap_from_brand_two_above+' can help you retake your previous rank. What if we could help you gain back market share from a competitor who has recently surpassed your brand?'
    ba.loc[(ba.rank_b > ba.rank_a) & (ba.rank_gap==3),'hook_5'] = 'Did you know that '+ba.rev_three_above_tagbrand+' took the #'+ (ba.rank_b-3).apply(lambda x: str(x)) + ' spot from you in '+ a_quarter_name + ' this year in the '+category+' category? Increasing quarterly sales by $'+ba.rev_gap_from_brand_three_above+' can help you retake your previous rank. What if we could help you gain back market share from a competitor who has recently surpassed your brand?'
    
    ### hook 6 ###
    
    ba.loc[(ba.rank_b > 1),'hook_6'] = 'You are currently ranked #'+ba.rank_b.apply(lambda x: str(x))+' in the '+category+' category in '+b_quarter_name+'. With our ecomm-focused data and insights, we can help your brand take the #'+ba.rank_b.apply(lambda x: str(x-1))+' spot from '+ba.rev_above_tagbrand+' by increasing your market share by '+(ba.market_share_above_val.astype(float) - ba.market_share_b_perc).apply(lambda x: str(round(x,2)))+'%.'

    ### hook 7 ###

    ba['market_share_below_tagbrand'] = ba.tagbrand.shift(-1)
    ba['brand_growth_below'] = ba['brand_growth_perc'].shift(-1)
    ba['rev_below'] = ba.rev_b.shift(-1)
    ba['rev_gap_from_brand_below'] = ba['rev_b'] - ba['rev_below']
    ba['rev_gap_from_brand_below_form'] = ba['rev_gap_from_brand_below'].apply(lambda x: rev_format(x))
    
    ba.loc[(ba.brand_growth_below>0) & (ba.rev_gap_from_brand_below>0) & (ba.brand_growth_below > ba.brand_growth_perc),'hook_7'] = 'Are you aware that '+ba.market_share_below_tagbrand.apply(lambda x: str(x))+' experienced '+ba.brand_growth_below.apply(lambda x: str(x))+'%  growth in the '+category+' category last quarter? They narrowed the revenue gap between your brand and theirs to $'+ba.rev_gap_from_brand_below_form+ '. Clearcut Analytics makes it easy to track your closest competitors to ensure that you are always one step ahead.'

    ### hook 8 ###
    
    ba.loc[(ba.brand_growth_perc > 0)&(ba.brand_growth_perc < ba.category_growth_perc),'hook_8'] = 'Congrats on $'+ba.rev_b_formatted+ ' in '+ b_quarter_name+' revenue with '+ba.brand_growth_perc.apply(lambda x: str(x))+'% growth in the '+category+' category. However, are you aware that the category as a whole grew by '+ba.category_growth_perc.apply(lambda x: str(round(x,2)))+'%? Meaning your brand is still behind the curve by '+(ba.category_growth_perc - ba.brand_growth_perc).apply(lambda x: str(round(x,2)))+' percentage points compared to the category. ClearCut can help your brand stay ahead of the curve with our robust analytics and data-driven insights.'

    ### hook 9 ###
    
    ba.loc[ba.rank_b > 3,'hook_9'] = 'I noticed '+ba.tagbrand+' was ranked #'+ba.rank_b.apply(lambda x: str(x))+' by revenue last quarter in the '+category+' category. Would you like to know which brands are ranked ahead of you in the #'+(ba.rank_b-1).apply(lambda x: str(x))+', #'+(ba.rank_b-2).apply(lambda x: str(x))+', & #'+ (ba.rank_b-3).apply(lambda x: str(x))+' spot? I would love the opportunity to show you our analytics portal that showcases your closest competitors in the category by sales revenue, market share, and more.'
    
    ba = ba[['tagbrand',
             'hook_1',
             'hook_2',
             'hook_3',
             'hook_4',
             'hook_5',
             'hook_6',
             'hook_7',
             'hook_8',
             'hook_9',
            'rev_b',
            'rev_a',
            'rev_b_formatted',
            'rev_a_formatted',
            'market_share_b_perc',
            'market_share_a_perc',
            'category_growth_perc',
            'brand_growth_perc',
            'growth_gap',
            'market_share_perc_change',
            'rank_b',
            'rank_a']]
    ba.columns = ['Brand',
                 'Hook 1',
                 'Hook 2',
                 'Hook 3',
                 'Hook 4',
                 'Hook 5',
                 'Hook 6',
                 'Hook 7',
                 'Hook 8',
                 'Hook 9',
                 'Revenue '+b_quarter_name,
                 'Revenue '+a_quarter_name,
                 'Revenue '+b_quarter_name+' Formatted',
                 'Revenue '+a_quarter_name+' Formatted',
                 'Market Share (%)'+b_quarter_name,
                 'Market Share (%)'+a_quarter_name,
                 'Category Growth (%)'+a_quarter_name+b_quarter_name,
                 'Brand Growth (%)'+a_quarter_name+b_quarter_name,
                 'Brand-Category Growth Gap',
                 'Market Share % Change',
                 'Rank in'+b_quarter_name,
                 'Rank in'+a_quarter_name
                 ]
    
    return ba



In [ ]:
df1 = calculate_fields('Q1Q2')

In [ ]:
df1.to_csv('/Users/avacheevers/Desktop/clearcut_categories/'+category+'_hooks_v4.csv')

In [ ]:
df1['Hook 2'].isna().value_counts()